In [7]:
from guess_number import guess_number
from numpy import mean, random
from math import log2, ceil

def average_int_result(func, repeat: int=1000, print_chart: bool=True, bar_len: int=35):
    def dec_func(*args, **kwargs):
        results = [func(*args, **kwargs) for i in range(repeat)]
        avrg_res = round(mean(results))
        
        if print_chart:
            # Printing a simple chart
            min_res = min(results) # lower bound of chart value axis
            max_res = max(results) # upper bound of chart value axis
            # Creating a dictionary:
            # keys are results returned by func();
            # values are numbers of times each result was returned.
            results = {i: results.count(i) 
                        for i in range(min_res, max_res+1)}
            max_val = max(results.values()) # count of the most frequent result
            
            for result in results.items():
                if result[1]:
                    chart_ch = '=' if result[0] == avrg_res else '-'
                    indent = ' '*(len(str(max_res)) - len(str(result[0])))
                    chart_bar = indent + chart_ch * ceil(result[1]/max_val * bar_len)
                    print(result[0], chart_bar, f"{result[1]} times")
                
        return avrg_res
    return dec_func

guess_number_chart = average_int_result(guess_number, print_chart=True)
print(f"Average number of tries is {guess_number_wrap(1,100)}")

guess_number = average_int_result(guess_number, print_chart=False)
for hi in [2**i for i in range(10,21)]:
    print(f"\n[1; 2**{int(log2(hi))}]")
    print(f"Average number of tries is {guess_number(1,hi)}")

1 - 7 times
2 --- 26 times
3 ----- 42 times
4 --------- 90 times
5 --------------- 147 times
6 =============================== 323 times
7 ----------------------------------- 365 times
Average number of tries is 6

[1; 2**10]
Average number of tries is 9

[1; 2**11]
Average number of tries is 10

[1; 2**12]
Average number of tries is 11

[1; 2**13]
Average number of tries is 12

[1; 2**14]
Average number of tries is 13

[1; 2**15]
Average number of tries is 14

[1; 2**16]
Average number of tries is 15

[1; 2**17]
Average number of tries is 16

[1; 2**18]
Average number of tries is 17

[1; 2**19]
Average number of tries is 18

[1; 2**20]
Average number of tries is 19
